In [1]:
%load_ext autoreload
%autoreload 2

import os
import shutil
import pickle
import time
import pprint
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from IPython.display import SVG

from pydrake.examples import QuadrotorGeometry
from pydrake.geometry import MeshcatVisualizer, Rgba, StartMeshcat
from pydrake.geometry.optimization import HPolyhedron, VPolytope
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser
from pydrake.perception import PointCloud
from pydrake.solvers import GurobiSolver,  MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder

from gcs.bezier import BezierGCS
from reproduction.uav.helpers import FlatnessInverter
from reproduction.uav.building_generation import *
from reproduction.util import *

gurobi_license = GurobiSolver.AcquireLicense()
mosek_license = MosekSolver.AcquireLicense()

In [2]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

meshcat.SetProperty("/Grid", "visible", False)
meshcat.SetProperty("/Axes", "visible", False)
meshcat.SetProperty("/Lights/AmbientLight/<object>", "intensity", 0.8)
meshcat.SetProperty("/Lights/PointLightNegativeX/<object>", "intensity", 0)
meshcat.SetProperty("/Lights/PointLightPositiveX/<object>", "intensity", 0)

INFO:drake:Meshcat listening for connections at http://localhost:7000


# Generate Building and Plan Through

In [3]:
start = np.array([-1, -1])
goal = np.array([2, 1])
building_shape = (3, 3)
start_pose = np.r_[(start-start)*5, 1.]
goal_pose = np.r_[(goal-start)*5., 1.]

# Generate a random building
np.random.seed(3)
grid, outdoor_edges, wall_edges = generate_grid_world(shape=building_shape, start=start, goal=goal)
regions = compile_sdf(FindModelFile("models/room_gen/building.sdf"), grid, start, goal, outdoor_edges, wall_edges)

In [4]:
# Build the GCS optimization
gcs = BezierGCS(regions, order=7, continuity=4, hdot_min=1e-3, full_dim_overlap=True)

gcs.addTimeCost(1e-3)
gcs.addPathLengthCost(1)
gcs.addVelocityLimits(-10 * np.ones(3), 10 * np.ones(3))
regularization = 1e-3
gcs.addDerivativeRegularization(regularization, regularization, 2)
gcs.addDerivativeRegularization(regularization, regularization, 3)
gcs.addDerivativeRegularization(regularization, regularization, 4)
gcs.addSourceTarget(start_pose, goal_pose, zero_deriv_boundary=3)

gcs.setPaperSolverOptions()
gcs.setSolver(MosekSolver())

# Solve GCS
trajectory = gcs.SolvePath(True, verbose=False, preprocessing=False)[0]

INFO:drake:Solved GCS shortest path using Mosek with convex_relaxation=true and preprocessing=false and no rounding.


Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 17681           
  Affine conic cons.     : 2257            
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 6290            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 3845
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 650
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 1               
Presolve 

INFO:drake:Solved GCS shortest path using Mosek with convex_relaxation=true and preprocessing=false and rounding.


6.672621236e+01   5.5e-05  0.73  
21  4.1e-02  1.1e-04  8.6e-04  1.06e+00   6.686189885e+01   6.673087920e+01   4.1e-05  0.75  
22  2.2e-02  6.0e-05  3.2e-04  1.07e+00   6.679610283e+01   6.672894808e+01   2.2e-05  0.77  
23  1.7e-02  4.6e-05  2.2e-04  1.09e+00   6.677857901e+01   6.672732927e+01   1.7e-05  0.80  
24  6.0e-03  4.3e-05  4.4e-05  1.09e+00   6.673065760e+01   6.671315669e+01   6.0e-06  0.82  
25  3.7e-03  2.6e-05  2.1e-05  1.10e+00   6.671719010e+01   6.670668268e+01   3.7e-06  0.84  
26  1.9e-03  3.4e-05  7.4e-06  1.07e+00   6.670986571e+01   6.670466645e+01   1.9e-06  0.86  
27  1.4e-03  2.6e-05  4.7e-06  1.03e+00   6.670832452e+01   6.670449646e+01   1.4e-06  0.88  
28  4.8e-04  7.3e-06  9.5e-07  1.03e+00   6.670546068e+01   6.670413322e+01   4.7e-07  0.91  
29  3.0e-04  4.2e-06  4.6e-07  1.01e+00   6.670505247e+01   6.670422911e+01   2.9e-07  0.93  
30  1.7e-04  2.4e-06  2.0e-07  1.00e+00   6.670475996e+01   6.670428655e+01   1.7e-07  0.97  
31  3.1e-05  4.4e-07  1.5e

INFO:drake:Finished 100 rounding trials.


g terminated.
Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 17681           
  Affine conic cons.     : 2257            
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 6290            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 16              
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 14814
Optimizer  - Cones                  : 1812
Optimizer  - Scalar variables       : 18097             conic                  : 7311            
Optimizer  - Semi-definite variables: 0                 scalarized             : 0               
Factor     - setup time             : 0.21              dense det. time        : 0.09            
Factor     - ML order time  

# Visualized Trajectory

In [5]:
view_regions = False
track_uav = False

# Build and run Diagram
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

parser = Parser(plant, scene_graph)
parser.package_map().Add("gcs", GcsDir())
model_id = parser.AddModelFromFile(FindModelFile("models/room_gen/building.sdf"))

plant.Finalize()

meshcat_cpp = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

animator = meshcat_cpp.StartRecording()
if not track_uav:
    animator = None
traj_system = builder.AddSystem(FlatnessInverter(trajectory, animator))
quad = QuadrotorGeometry.AddToBuilder(builder, traj_system.get_output_port(0), scene_graph)

diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)

meshcat.Delete()

if view_regions:
    for ii in range(len(regions)):
        v = VPolytope(regions[ii])
        meshcat.SetTriangleMesh("iris/region_" + str(ii), v.vertices(),
                                ConvexHull(v.vertices().T).simplices.T, Rgba(0.698, 0.67, 1, 0.4))
        
# Simulate
end_time = trajectory.end_time()
simulator.AdvanceTo(end_time+0.05)
meshcat_cpp.PublishRecording()

INFO:drake:PackageMap: Downloading https://github.com/RobotLocomotion/models/archive/611246c443152946e9dcc901b4f956d89a439a61.tar.gz


In [6]:
with open (os.path.join(GcsDir(), "data/uav_example/trajectory.html"), "w") as f:
    f.write(meshcat.StaticHtml())